In [1]:
import pandas as pd
from numpy import poly1d
from matplotlib import pyplot as plt
import numpy as np

In [2]:
x = [0.17453, 0.5236, 0.87267, 1.22173, 1.5708, 1.91986, 2.26893]
y = [0.00162, 0.00252, 0.00498, 0.0129, 0.03964, 0.1207, 0.34097]

In [3]:
def newton_poly(b, x0, a):
    poly = b[0] + b[1]*(x0 - a[0]) + b[2]*(x0 - a[0])*(x0 - a[1]) + b[3]*(x0 - a[0])*(x0 - a[1])*(x0 - a[2])
    poly += b[4]*(x0 - a[0])*(x0 - a[1])*(x0 - a[2])*(x0 - a[3])
    poly += b[5]*(x0 - a[0])*(x0 - a[1])*(x0 - a[2])*(x0 - a[3])*(x0 - a[4])
    poly += b[6]*(x0 - a[0])*(x0 - a[1])*(x0 - a[2])*(x0 - a[3])*(x0 - a[4])*(x0 - a[5])
    return poly

In [4]:
b0 = y
b1 = []
for i in range(1, 7):
    b1.append((y[i] - y[i-1])/(x[i] - x[i-1]))
b2 = []
for i in range(1, 6):
    b2.append((b1[i] - b1[i-1])/(x[i+1] - x[i-1]))
b3 = []
for i in range(1, 5):
    b3.append((b2[i] - b2[i-1])/(x[i+2] - x[i-1]))
b4 = []
for i in range(1, 4):
    b4.append((b3[i] - b3[i-1])/(x[i+3] - x[i-1]))
b5 = []
for i in range(1, 3):
    b5.append((b4[i] - b4[i-1])/(x[i+4] - x[i-1]))
b6 = (b5[1] - b5[0])/(x[6] - x[0])

In [5]:
b = [b0[0], b1[0], b2[0], b3[0], b4[0], b5[0], b6]

In [6]:
print(b)

[0.00162, 0.0025782794281949185, 0.00640132018726596, 0.015283186066176081, 0.026546727555321226, 0.020395660881294617, 0.01117456557993848]


In [8]:
newton_coefs = [0 for _ in range(len(x))]
newton_coefs[0] += b[0]

for i in range(1, len(x)):
    n_poly = poly1d(x[:i], True)
    npc = n_poly.coef    
    k = len(npc) - 1
    j = 0
    while k > -1:
        newton_coefs[k] += b[i] * npc[j]
        j += 1
        k -= 1 
print(newton_coefs)

[0.003284559824898437, -0.019391458207426944, 0.07479615193127234, -0.1227016196709355, 0.1094543433033508, -0.04981625782491904, 0.01117456557993848]


In [9]:
newton_coefs.reverse()
p1 = poly1d(newton_coefs).deriv(1)
p1_a = [p1(el) for el in x]
spline_coefs = []

def find_spline_coeffs(x, f, P):
    den = (x[1] - x[0]) ** 3

    a_3 = ((P[1] + P[0]) * (x[1] - x[0]) - 2 * (f[1] - f[0])) / den
    a_2 = (3 * (f[1] - f[0]) * (x[1] + x[0]) - (P[1] * (x[1] + 2 * x[0]) + P[0] * (x[0] + 2 * x[1])) * (
            x[1] - x[0])) / den
    a_1 = ((P[1] * x[0] * (2 * x[1] + x[0]) + P[0] * x[1] * (x[1] + 2 * x[0])) * (x[1] - x[0]) - 6 * (f[1] - f[0]) * x[
        0] * x[1]) / den
    a_0 = ((f[1] * x[0] ** 2 * (3 * x[1] - x[0]) + f[0] * x[1] ** 2 * (x[1] - 3 * x[0])) - (
            P[1] * x[0] ** 2 * x[1] + P[0] * x[0] * x[1] ** 2) * (x[1] - x[0])) / den

    return [a_3, a_2, a_1, a_0]



In [10]:
for i in range(len(x) - 1):
    coeffs = find_spline_coeffs([x[i], x[i + 1]], [y[i], y[i + 1]], [p1_a[i], p1_a[i + 1]])
    spline_coefs.append(coeffs)


In [11]:
print(spline_coefs)

[[-0.022677772893361245, 0.034018688384902636, -0.01219075339547275, 0.002831980479998416], [0.016016487010490497, -0.024259505450194906, 0.01701326919457167, -0.002036379520059403], [0.0473792122671172, -0.10784634086483634, 0.0912475906847518, -0.02400582330078225], [0.12844509879497898, -0.4104293926726042, 0.4675944641038418, -0.17998773924572278], [0.31624730169393356, -1.3048699923716183, 1.8874124643600936, -0.9311698019718887], [0.6678201991784162, -3.343208122854888, 5.826512158975495, -3.468505123912762]]
